# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [13]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [14]:
ws = Workspace.from_config()

print('WORKSPACE DETAILS:\nWorkspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'Heart_Disease_Kaggle_AutoML'

experiment=Experiment(ws, experiment_name)
print("Experiment created")

run = experiment.start_logging()

WORKSPACE DETAILS:
Workspace name: quick-starts-ws-136809
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-136809
Experiment created


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Creating a compute cluster
# vm_size = "Standard_D2_V2"
# max_nodes = 4.

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Exploring the Dataset

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd

url_path = "https://raw.githubusercontent.com/bharati-21/AZMLND-Capstone-Project/master/files/heart.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

df = pd.read_csv(url_path)

In [ ]:
# ds_dataframe = ds.to_pandas_dataframe()
df.head()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)
x.head()

In [ ]:
import os
path = "./Data"
try:
    os.makedirs(path, exist_ok=True)
    print("Dicrectory 'Data' created...")
except OSError:
    print("Directory 'Data' cannot be created...")

In [ ]:
# split data into test and train sets

from sklearn.model_selection import train_test_split
import pandas as pd

#spliting the data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state=40)
print(test_x.shape, train_x.shape, test_y.shape, train_y.shape)

train_df = train_x.copy(deep=True)
train_df['target'] = train_y
print(train_df.shape)

test_df = test_x.copy(deep=True)
test_df['target'] = test_y
print(test_df.shape)

# saving test and train data locally
train_df.to_csv("./Data/train_data.csv")
test_df.to_csv("./Data/test_data.csv")

In [ ]:
deafult_store = ws.get_default_datastore()
deafult_store.upload(src_dir="Data", target_path="Data")

In [ ]:
from azureml.core import Dataset

train_data = TabularDatasetFactory.from_delimited_files(path=[(deafult_store, ('Data/train_data.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(deafult_store, ('Data/test_data.csv'))])


In [ ]:
train_data.to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here

automl_config = AutoMLConfig (
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name="target",
    n_cross_validations=5,
    enable_onnx_compatible_models=True,
    compute_target = cpu_cluster)

print("AutoML config created.")

In [ ]:
from azureml.widgets import RunDetails

# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
#from azureml.widgets import RunDetails

#RunDetails(remote_run).show()
#remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)

In [ ]:
print(fitted_model)

In [ ]:
best_run.get_tags()

In [ ]:
import joblib
from azureml.core.model import Model

#TODO: Save the best model

description = "Best Model from AutoML run of the Cleveland Foundation Clinic Heart Disease Dataset"
automl_model = remote_run.register_model(model_name='heart_disease_autoML_best_model.pkl', description=description)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service